Teniendo en cuenta los resultados y las conclusiones de los anteriores notebooks, normalizaremos las variables Tempo, Loudness y Duration_ms y transformaremos a logaritmo el target Streams


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

train_df = pd.read_csv('spotify_limpio_train.csv')

train_df.head()

,Artist,Track,Album,Album_type,Stream,Danceability,Energy,Key,Key_name,Loudness,Speechiness,Acousticness,Liveness,Valence,Tempo,Tempo_category,Duration_ms,Licensed,official_video
0,KAYTRANADA,LITE SPOTS,99.9%,album,29240851.0,0.884,0.549,1,C# / Db,-11.683,0.4710,0.03460,0.112,0.394,120.461,Moderato / Allegretto,230920.0,True,True
1,Rita Ora,Follow Me,Follow Me,single,82813284.0,0.673,0.729,6,F# / Gb,-4.879,0.0496,0.08300,0.100,0.675,122.023,Allegro,169672.0,True,True
2,Big Sean,Bounce Back,I Decided.,album,666145000.0,0.780,0.575,1,C# / Db,-5.628,0.1390,0.10600,0.129,0.273,81.502,Andante,222360.0,True,True
3,Dave Matthews Band,#41,Crash,album,34528391.0,0.577,0.726,4,E,-8.011,0.0299,0.00199,0.158,0.764,107.416,Andante,399800.0,True,True
4,Ray Dalton,Call It Love - Klingande Remix,Call It Love (Klingande Remix),single,1153344.0,0.671,0.687,10,A# / Bb,-8.669,0.0502,0.01110,0.504,0.623,121.992,Allegro,149508.0,True,True


In [2]:
train_df = train_df.drop(columns=[ 'Artist', 'Track', 'Album', 'Album_type', 'Key_name', 'Tempo_category'])

In [3]:
train_df["log_stream"] = np.log1p(train_df["Stream"])

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

cols_to_scale = ["Loudness", "Tempo", "Duration_ms"]

train_df[cols_to_scale] = scaler.fit_transform(
    train_df[cols_to_scale]
)

In [5]:
train_df.head()

,Stream,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Liveness,Valence,Tempo,Duration_ms,Licensed,official_video,log_stream
0,29240851.0,0.884,0.549,1,-0.883776,0.4710,0.03460,0.112,0.394,-0.000111,0.054275,True,True,17.191077
1,82813284.0,0.673,0.729,6,0.591959,0.0496,0.08300,0.100,0.675,0.052744,-0.442334,True,True,18.232099
2,666145000.0,0.780,0.575,1,0.429507,0.1390,0.10600,0.129,0.273,-1.318414,-0.015131,True,True,20.317018
3,34528391.0,0.577,0.726,4,-0.087348,0.0299,0.00199,0.158,0.764,-0.441531,1.423583,True,True,17.357293
4,1153344.0,0.671,0.687,10,-0.230063,0.0502,0.01110,0.504,0.623,0.051695,-0.605827,True,True,13.958177


In [6]:
import pandas as pd

X_train = train_df.drop(columns=['Stream', 'log_stream'], axis=1)
y_train = train_df[ 'log_stream']

Ahora, vamos a aplicar las transformaciones que hemos considerado al conjusto de test. Estas son: Eliminar las columnas con strings y las que se saben una vez se saca la canción (Views, comments...) normalizar las variables Loudness, Tempo y Duration_ms y aplicar la transformacion logarítmica a Stream.

In [7]:
test_df = pd.read_csv('spotify_test.csv')

test_df.head()


,Unnamed: 0,Artist,Url_spotify,Track,Album,Album_type,Uri,Danceability,Energy,Key,...,Url_youtube,Title,Channel,Views,Likes,Comments,Description,Licensed,official_video,Stream
0,18320,Jessie Reyez,https://open.spotify.com/artist/3KedxarmBCyFBe...,FIGURES,Kiddo,album,spotify:track:3U3J5v3rkx89WnFEQvAJD5,0.600,0.13900,7.0,...,https://www.youtube.com/watch?v=wxLUj1Mrars,Jessie Reyez - Figures (Official Video),JessieReyezVEVO,105473489.0,1130146.0,23767.0,YESSIE – Out Now: https://JessieReyez.lnk.to/Y...,True,True,167619713.0
1,2828,Claude Debussy,https://open.spotify.com/artist/1Uff91EOsvd99r...,"Rêverie, L. 68",Debussy: Estampes; Pour le piano; 6 épigraphes...,album,spotify:track:086sjLPEqdKBgTxbTeCLCv,0.250,0.00904,5.0,...,https://www.youtube.com/watch?v=QRjllL-MP0U,Debussy - Rêverie,AleksandrSolzenicyn,15974768.0,107236.0,3080.0,"F. J. Thiollier, pianist",False,False,40399275.0
2,9119,Ben E. King,https://open.spotify.com/artist/3plJVWt88Eqjvt...,Spanish Harlem,Spanish Harlem,album,spotify:track:3UCmuRdeTriWgOZMEJsfqZ,0.608,0.59400,2.0,...,https://www.youtube.com/watch?v=DdZYUr-eJNc,Ben E King - Spanish Harlem (1961),YachtiCal Media,37204.0,572.0,23.0,Ben E King - Spanish Harlem (1961),False,False,18866293.0
3,6730,Kenny G,https://open.spotify.com/artist/6I3M904Y9IwgDj...,Sentimental,Breathless,album,spotify:track:5I7uF6AdRy5SbtHE6k1QJo,0.423,0.26900,8.0,...,https://www.youtube.com/watch?v=oHONFub5MPM,Kenny G - Sentimental,KennyGVEVO,15609999.0,57744.0,2695.0,Kenny G's official music video for 'Sentimenta...,True,True,10934323.0
4,2869,The Alchemist,https://open.spotify.com/artist/0eVyjRhzZKke2K...,Loose Change (feat. Earl Sweatshirt),This Thing Of Ours,album,spotify:track:2Z5MbKovyels3EXitye0eo,0.506,0.62800,9.0,...,https://www.youtube.com/watch?v=7z1WyqWaSYs,"The Alchemist - ""Loose Change"" feat. Earl Swea...",Al Chemist,1423560.0,62177.0,2552.0,Directed by Lonewolf\nfrom This Thing Of Ours ...,True,True,21176930.0


In [8]:
test_df = test_df.drop(columns=[ 'Unnamed: 0', 'Artist', 'Url_spotify', 'Track', 'Album', 'Album_type', 'Uri','Url_youtube', 'Title', 'Channel', 'Views', 'Likes',
       'Comments', 'Description', 'Instrumentalness'])

In [9]:
test_df["log_stream"] = np.log1p(test_df["Stream"])

In [10]:
scaler = StandardScaler()

cols_to_scale = ["Loudness", "Tempo", "Duration_ms"]

test_df[cols_to_scale] = scaler.fit_transform(
    test_df[cols_to_scale]
)

In [11]:
X_test = test_df.drop(columns=['Stream', 'log_stream'], axis=1)
y_test = test_df['log_stream']

Random forest ha sido el algoritmo seleccionado, porque nos daba las mejores métricas

Aqui estamos probando el algoritmo GridSearchCV para encontrar los mejores hiperparámetros

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Modelo base
rf = RandomForestRegressor(
    random_state=42,
    n_jobs=-1
)

# Espacio de hiperparámetros
param_grid = {
    "n_estimators": np.arange(100, 600, 100),
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10]
}

# Grid Search + validación cruzada
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)

# Entrenamiento
grid_search.fit(X_train, y_train)

# Mejor modelo
best_rf = grid_search.best_estimator_

print("Mejores hiperparámetros:")
print(grid_search.best_params_)

print(f"Mejor RMSE CV: {-grid_search.best_score_:.4f}")

A continuación vamos a usar el código RandomizedSearchCV para ver los mejores hiperparámetros, para tener otra alternativa y compararlos

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

rf = RandomForestRegressor(
    random_state=42,
    n_jobs=-1
)

param_dist = {
    "n_estimators": np.arange(100, 600, 100),
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10]
}

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=30,
    cv=5,
    scoring="neg_root_mean_squared_error",
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

best_rf = random_search.best_estimator_

print("Mejores hiperparámetros:")
print(random_search.best_params_)

print(f"Mejor RMSE CV: {-random_search.best_score_:.4f}")

Ya tenemos los mejores hiperparámetros, a continuación aplicamos el algoritmo Random forest, a todo train

In [12]:
from sklearn.model_selection import cross_val_score
import numpy as np


def val_cruz(model, X, y, n_splits=5, random_state=42):
    

    #cv = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)


    scores_rmse = cross_val_score(
        model,
        X,
        y,
        cv=n_splits,
        scoring="neg_root_mean_squared_error"
    )


    scores_mape = cross_val_score(
            model,
            X,
            y,
            cv=n_splits,
            scoring="neg_mean_absolute_percentage_error"
        )
        
    scores_mae = cross_val_score(
        model,
        X,
        y,
        cv=n_splits,
        scoring="neg_mean_absolute_error"
    )

    scores_r2 = cross_val_score(
        model,
        X,
        y,
        cv=n_splits,
        scoring="r2"
    )

    # Pasar a positivo
    scores_rmse = -scores_rmse
    scores_mae = -scores_mae
    scores_mape = -scores_mape 

    print(" Resultados validación cruzada")
    print("-" * 40)
    print(f"RMSE: {scores_rmse}")
    print(f"MAPE: {scores_mape}")
    print(f"MAE:  {scores_mae}")
    print(f"R²:   {scores_r2}")
    print("-" * 40)
    print(f"RMSE medio: {scores_rmse.mean():.4f} ± {scores_rmse.std():.4f}")
    print(f"MAPE medio:  {scores_mape.mean():.4f} ± {scores_mape.std():.4f}")
    print(f"MAE medio:  {scores_mae.mean():.4f} ± {scores_mae.std():.4f}")
    print(f"R² medio:   {scores_r2.mean():.4f} ± {scores_r2.std():.4f}")

    return scores_rmse.mean(), scores_rmse.std(), scores_mape.mean(), scores_mape.std(), scores_mae.mean(),scores_mae.std(),scores_r2.mean(),scores_r2.std()



Random forest

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

model = RandomForestRegressor(n_estimators = 500, max_depth = None, min_samples_split = 2, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE (log): {rmse:.3f}")
print(f"MAE (log): {mae:.3f}")
print(f"R²: {r2:.3f}")


RMSE (log): 1.517
MAE (log): 1.149
R²: 0.196


Deshacemos la transformación del logaritmo.

In [15]:
y_test_real = np.exp(y_test)
y_pred_real = np.exp(y_pred)

rmse_real = np.sqrt(mean_squared_error(y_test_real, y_pred_real))
mae_real = mean_absolute_error(y_test_real, y_pred_real)

print(f"RMSE (streams): {rmse_real:,.0f}")
print(f"MAE (streams): {mae_real:,.0f}")


RMSE (streams): 254,351,974
MAE (streams): 113,033,760
